In [114]:
import numpy as np
import pandas as pd

In [118]:
df = pd.read_csv('example_data.csv')
df

,humility,outlook,temp,windy,play
0,high,sunny,hot,False,no
1,high,sunny,hot,True,no
2,high,overcast,hot,False,yes
3,high,rainy,mild,False,yes
4,normal,rainy,cool,False,yes
5,normal,rainy,cool,True,no
6,normal,overcast,cool,True,yes
7,high,sunny,mild,False,no
8,normal,sunny,cool,False,yes
9,normal,rainy,mild,False,yes


In [137]:
d=np.array(df.values)
label=[]
for sample in d:
    if not sample[-1] in label:
        label.append(sample[-1])
print(label)
d_train=d[:9]
d_test=d[9:]
d

['no', 'yes']


array([['high', 'sunny', 'hot', False, 'no'],
       ['high', 'sunny', 'hot', True, 'no'],
       ['high', 'overcast', 'hot', False, 'yes'],
       ['high', 'rainy', 'mild', False, 'yes'],
       ['normal', 'rainy', 'cool', False, 'yes'],
       ['normal', 'rainy', 'cool', True, 'no'],
       ['normal', 'overcast', 'cool', True, 'yes'],
       ['high', 'sunny', 'mild', False, 'no'],
       ['normal', 'sunny', 'cool', False, 'yes'],
       ['normal', 'rainy', 'mild', False, 'yes'],
       ['normal', 'sunny', 'mild', True, 'yes'],
       ['high', 'overcast', 'mild', True, 'yes'],
       ['normal', 'overcast', 'hot', False, 'yes'],
       ['high', 'rainy', 'mild', True, 'no']], dtype=object)

In [141]:
data = {}
for i in d_train:
    if not i[-1] in data:
        data[i[-1]] = [i]
    else:
        data[i[-1]].append(i)

data_test={}
for i in d_test:
    if not i[-1] in data_test:
        data_test[i[-1]] = [i]
    else:
        data_test[i[-1]].append(i)
print(data)
print(data_test)

{'no': [array(['high', 'sunny', 'hot', False, 'no'], dtype=object), array(['high', 'sunny', 'hot', True, 'no'], dtype=object), array(['normal', 'rainy', 'cool', True, 'no'], dtype=object), array(['high', 'sunny', 'mild', False, 'no'], dtype=object)], 'yes': [array(['high', 'overcast', 'hot', False, 'yes'], dtype=object), array(['high', 'rainy', 'mild', False, 'yes'], dtype=object), array(['normal', 'rainy', 'cool', False, 'yes'], dtype=object), array(['normal', 'overcast', 'cool', True, 'yes'], dtype=object), array(['normal', 'sunny', 'cool', False, 'yes'], dtype=object)]}
{'yes': [array(['normal', 'rainy', 'mild', False, 'yes'], dtype=object), array(['normal', 'sunny', 'mild', True, 'yes'], dtype=object), array(['high', 'overcast', 'mild', True, 'yes'], dtype=object), array(['normal', 'overcast', 'hot', False, 'yes'], dtype=object)], 'no': [array(['high', 'rainy', 'mild', True, 'no'], dtype=object)]}


计算信息熵

In [142]:
def Ent(data):
    """
    
    :param data: dict
    :return: 信息熵
    """
    num=sum(len(data[i]) for i in data)
    res=0
    for i in data:
        pi=len(data[i])/num
        res+=pi*np.log(pi)
    return -res

分割前的信息熵

In [143]:
E0=Ent(data)
E0

0.6869615765973234

In [144]:
def split_dataframe(data, col):
    """
    根据第col列属性分割后的数据集
    :param data: dict
    :param col:
    :return: dict
    """
    res={}
    for types in data:
        for sample in data[types]:
            if not sample[col] in res:
                res[sample[col]]={}
                res[sample[col]][sample[-1]]=[sample]
            else:
                if not sample[-1] in res[sample[col]]:
                    res[sample[col]][sample[-1]]=[sample]
                else:
                    res[sample[col]][sample[-1]].append(sample)
    return res
    

In [145]:
split_play=split_dataframe(data,2)
split_play

{'hot': {'no': [array(['high', 'sunny', 'hot', False, 'no'], dtype=object),
   array(['high', 'sunny', 'hot', True, 'no'], dtype=object)],
  'yes': [array(['high', 'overcast', 'hot', False, 'yes'], dtype=object)]},
 'cool': {'no': [array(['normal', 'rainy', 'cool', True, 'no'], dtype=object)],
  'yes': [array(['normal', 'rainy', 'cool', False, 'yes'], dtype=object),
   array(['normal', 'overcast', 'cool', True, 'yes'], dtype=object),
   array(['normal', 'sunny', 'cool', False, 'yes'], dtype=object)]},
 'mild': {'no': [array(['high', 'sunny', 'mild', False, 'no'], dtype=object)],
  'yes': [array(['high', 'rainy', 'mild', False, 'yes'], dtype=object)]}}

信息增益

In [146]:
def Gain(Class,subClass):
    """
    根据某属性分支后的信息增益
    :param Class: dict of lables 
    :param subClass: dict with multi-2d arrays
    :return:
    """
    e=Ent(Class)
    num=sum(len(Class[i]) for i in Class)
    temp=0
    for node in subClass:
        cur_data=subClass[node]
        e_sub=Ent(cur_data)
        num_node=sum(len(cur_data[i]) for i in cur_data)
        temp+=(num_node/num)*e_sub
    return e-temp

In [147]:
Gain_play=Gain(data,split_play)
Gain_play

0.07083074943292766

In [148]:
def choose_best_col(data, label):
    """

    :param data: dict with key: labels
    :param label: label list
    :return: the col of best feature and maxGain
    """
    best_feature, maxGain = -1, -1
    # 遍历每个feature col
    for feature in range(len(data[label[0]][0]) - 1):
        split_feature = split_dataframe(data, feature)
        Gain_feature = Gain(data, split_feature)
        if Gain_feature > maxGain:
            best_feature = feature
            maxGain = Gain_feature

    return best_feature, maxGain

In [149]:
best_feature,maxGain=choose_best_col(data,label)
print('best feature is:',best_feature)
maxGain

best feature is: 1


0.2248634562240266

构造树

In [150]:
class ID3Tree:

    def majorityVote(self,data):
        maxLabel,maxCount=-1,-1
        for i in data:
            if len(data[i])>maxCount:
                maxLabel=i
                maxCount=len(data[i])
        return maxLabel


    def buildTree(self,data,label):
        # 如果当前组只有一种label，返回当前label
        if len(data)==1:
            return self.majorityVote(data)
        best_feature,maxGain=choose_best_col(data,label)
        split_data=split_dataframe(data,best_feature)
        Tree = {best_feature:{}}
        for sub in split_data:
            Tree[best_feature][sub]=self.buildTree(split_data[sub],label)
        return Tree

In [155]:
os=ID3Tree()
Tree=os.buildTree(data,label)
print(Tree)

{1: {'sunny': {0: {'high': 'no', 'normal': 'yes'}}, 'rainy': {3: {True: 'no', False: 'yes'}}, 'overcast': 'yes'}}


预测测试集

In [168]:
def predict(Tree,data_test,label):
    res=[]
    for sample in data_test:
        temp=Tree
        while True:
            feature=list(temp.keys())
            feature=feature[0]
            temp=temp[feature]
            pred=temp[sample[feature]]
            if pred in label:
                res.append(pred)
                break
            else:
                temp=pred
    return res
        

测试集很简单，accuracy=100%

In [170]:
predict(Tree,d_test,label)

['yes', 'yes', 'yes', 'yes', 'no']